# Advanced Lane Finding
## Udacity Self Driving Car - Project 4

Lots of work to do here but the pipeline is in run-able state

Can run all cells, will process default video, see last cells

In [1]:
# fix for pyqt5
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QMenu, QVBoxLayout, QSizePolicy, QMessageBox, QWidget, QPushButton
from PyQt5.QtGui import QIcon
#******************************************************************
#import matplotlib
#matplotlib.use("Qt5Agg")
#******************************************************************
from PyQt5 import QtCore 
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
import matplotlib.pyplot as plt
%matplotlib inline

import math
import numpy as np
import cv2
from cv2 import erode
from cv2 import dilate
import glob

import pickle


In [25]:
PRINT_OUTS = False
PARALLEL_THRESHOLD = (0.0004, 0.36)
LANE_WIDTH_THRESHOLD = (400,650)
CURVATURE_THRESHOLD = (400, 100) # (Error betwen left line and right line, Min curvature permitted, metres (based on U.S design rules))


In [42]:
class Lane():
    def __init__(self, n_fits=1):
        # how many frames should be collected for merging
        self.n_fits = n_fits
        # keep a list of previous warped binary images 
        self.image_list = []
        # the best frame to use for processing lane lines
        self.combined_lane_img = None
        
    def add_lane_img(self, warped_lane_binary):
        # Add the warped lane binary to the list of previous binary images
        self.image_list.insert(0,warped_lane_binary)
        # If the queue is full, remove the oldest
        if len(self.image_list)>self.n_fits:
            self.image_list.pop() 
    
    def get_combined_lane_img(self, warped_lane_binary):
        self.add_lane_img(warped_lane_binary)
        # Now we update the best image to use for fitting
        if self.combined_lane_img is not None:
            combined_binary = np.zeros_like(self.combined_lane_img)
            for image in self.image_list:
                combined_binary[ (combined_binary == 1) | (image == 1) ] = 1
            self.combined_lane_img = combined_binary
        else:
            self.combined_lane_img = warped_lane_binary
        return self.combined_lane_img

    
    # Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self, n_fits=1):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
        # the number of frames to average
        self.n_fits = n_fits

                        
    def update_line(self, detected, current_fit, radius_of_curvature, line_base_pos, allx, ally):
        self.detected = detected
        #if detected:
        self.current_fit = current_fit
        self.radius_of_curvature = radius_of_curvature
        self.line_base_pos = line_base_pos
        self.allx = allx
        self.ally = ally

        # Add the current fit to the list of previous fits
        self.recent_xfitted.insert(0,current_fit)
        # If the queue is full, remove the oldest
        if len(self.recent_xfitted)>self.n_fits:
            self.recent_xfitted.pop() 

        # Calculate the current fit with latest x and y values
        #print("before",self.current_fit)
        #self.current_fit = np.polyfit(self.allx, self.ally, 2)

        if self.best_fit is None:
            self.best_fit = self.current_fit
        else:
            # add up all the previous best_fits with the new one, take the average.
            self.best_fit = (((self.n_fits-1) * self.best_fit) + current_fit)/self.n_fits
        
    def get_best_fit(self, current_fit):
        old_fit = self.best_fit
        if PRINT_OUTS:
                print("Current Fit:",current_fit)
                print("Old Fit:",old_fit)
                print("nFits:",self.n_fits)
        if self.best_fit == []:
            self.best_fit = current_fit
            if PRINT_OUTS:
                print("Best Fit Empty now Initialised to Current")
        else:
            # add up all the previous best_fits with the new one, take the average.
            new_fit = (((self.n_fits-1) * old_fit) + current_fit)/self.n_fits
            if PRINT_OUTS:
                print("Best Fit Averaged")
        if PRINT_OUTS:
                print("New Fit:",new_fit)
        self.best_fit = new_fit
        return self.best_fit
        
 

In [4]:
camera_calibration = "camera_cal/camera_cal.p"
def fetch_camera_calibration():
    with open(camera_calibration, "rb" ) as f:
        ret = pickle.load(f)
        mtx = pickle.load(f)
        dist = pickle.load(f)
        rvecs = pickle.load(f)
        tvecs = pickle.load(f)
    return ret, mtx, dist, rvecs, tvecs

def fetch_perspective_transform():
    with open("camera_cal/perspective_cal.p", "rb" ) as f:
        M = pickle.load(f)
        Minv = pickle.load(f)
    return M, Minv

In [5]:
def remove_distortion(dist_img, cameraMatrix, distCoeffs):
    return cv2.undistort(dist_img, cameraMatrix, distCoeffs, None, cameraMatrix)

In [6]:
# When apply colour mask, lets only weight the areas we suspect to see the lanes
X_COL_OFFSET = 60
Y_COL_OFFSET = 90

def region_of_interest(img, x_offset, y_offset):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    
    # First define four points to form a polygon    
    imshape = img.shape
    vertices = np.array([[(0,imshape[0]),
                          (imshape[1]/2-x_offset, imshape[0]/2+y_offset),
                          (imshape[1]/2+x_offset, imshape[0]/2+y_offset),
                          (imshape[1],imshape[0])]],
                            dtype=np.int32)
    
    #define a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel colour to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_colour = (255,) * channel_count
    else:
        ignore_mask_colour = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill colour    
    cv2.fillPoly(mask, vertices, ignore_mask_colour)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def gaussian_blur(img, kernel_size=5):
    """Applies a Gaussian Noise kernel"""
    # Default size of 5
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)


# Define a function that takes an image, gradient orientation,
# and threshold min / max values.
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Convert to grayscale
    if len(img.shape) == 3: 
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1

    # Return the result
    return binary_output

# Define a function to return the magnitude of the gradient
# for a given sobel kernel size and threshold values
def mag_thresh(img, sobel_kernel=3, thresh=(0, 255)):
    # Convert to grayscale
    if len(img.shape) == 3: 
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= thresh[0]) & (gradmag <= thresh[1])] = 1

    # Return the binary image
    return binary_output

# Define a function to threshold an image for a given range and Sobel kernel
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    if len(img.shape) > 2: 
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output

def binary_threshold(img, thresh=(0,255)):
    binary = np.zeros_like(img)
    binary[(img > thresh[0]) & (img <= thresh[1])] = 1
    #binary = gaussian_blur(binary, kernel_size=blur_kernal_size)
    return binary

#def dir_pipe(img, dir_kernel=32, dir_thresh=(0.8,1.1), :
#    dir_binary = dir_threshold(img, sobel_kernel=31, thresh=(0.80, 1.1))
#    dst_thresh = (0.6, 1)
#    dst_binary = np.zeros_like(dst)
#    dst_binary[(dst > dst_thresh[0]) & (dst <= dst_thresh[1])] = 1
#    return region_of_interest(dst_binary,  X_COL_OFFSET, Y_COL_OFFSET)

#def preprocess(img, s_threshold=(210,255), r_threshold=(210,255), l_threshold=(210,255), b_threshold=(148,255), mask=True):
def preprocess(img, s_threshold=(230,255), r_threshold=(235,255), l_threshold=(235,255), b_threshold=(185,255), mask=True):
    #img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Lab
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l_channel = lab[:,:,0]
    b_channel = lab[:,:,2]
    l_thresh = binary_threshold(l_channel, thresh=l_threshold)
    b_thresh = binary_threshold(b_channel, thresh=b_threshold)
    
    #HLS
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS).astype(np.float)
    s_channel = hls[:,:,2]
    r_channel = img[:,:,0]
    s_thresh = binary_threshold(s_channel, thresh=s_threshold)
    r_thresh = binary_threshold(r_channel, thresh=r_threshold)
    
    # Comine all
    combined_binary = np.zeros_like(l_thresh)
    combined_binary[ (l_thresh == 1) | (b_thresh == 1) | (s_thresh == 1) | (r_thresh == 1) ] = 1
    
    if mask == True:
        combined_binary = region_of_interest(combined_binary,  X_COL_OFFSET, Y_COL_OFFSET)
        
    return  combined_binary
    

In [7]:
%matplotlib notebook
from matplotlib.widgets import Button
import matplotlib.pyplot as plt
        
def get_pt(event, x, y, flags, param):
    # grab references to the global variables
    global corners, new_pt

    # if the left mouse button was clicked, add (x, y) to coordinate list
    if event == cv2.EVENT_LBUTTONUP:
        corners.append((x, y))
        new_pt = True;

def select_lane_corners(image):
    global new_pt, corners
    new_pt = False
    corners = []
    
    # load the image and setup the mouse callback function
    clone = image.copy()
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", get_pt)

    # keep looping until the 4 coords received
    # r to reset
    # c to cancel
    cv2.imshow("image", clone)
    while len(corners)<4:

        # display the image and wait for a keypress

        key = cv2.waitKey(1) & 0xFF

        # if the 'r' key is pressed, reset the cropping region
        if key == ord("r"):
            clone = image.copy()
            corners = []
        # if the 'c' key is pressed, break from the loop
        elif key == ord("c"):
            break

        if new_pt == True:
            cv2.line(clone, (0,corners[len(corners)-1][1]), (image.shape[1],corners[len(corners)-1][1]), (0,0, 255), 1)
            new_pt = False

        cv2.imshow("image", clone)

    # close all open windows
    cv2.destroyAllWindows()
    

In [8]:
# http://www.pyimagesearch.com/2016/03/21/ordering-coordinates-clockwise-with-python-and-opencv/
from scipy.spatial import distance
 
def order_points(pts):
    # sort the points based on their x-coordinates
    xSorted = pts[np.argsort(pts[:, 0]), :]

    # grab the left-most and right-most points from the sorted
    # x-roodinate points
    leftMost = xSorted[:2, :]
    rightMost = xSorted[2:, :]

    # now, sort the left-most coordinates according to their
    # y-coordinates so we can grab the top-left and bottom-left
    # points, respectively
    leftMost = leftMost[np.argsort(leftMost[:, 1]), :]
    (tl, bl) = leftMost

    # now that we have the top-left coordinate, use it as an
    # anchor to calculate the Euclidean distance between the
    # top-left and right-most points; by the Pythagorean
    # theorem, the point with the largest distance will be
    # our bottom-right point
    D = distance.cdist(tl[np.newaxis], rightMost, "euclidean")[0]
    (br, tr) = rightMost[np.argsort(D)[::-1], :]

    # return the coordinates in top-left, top-right,
    # bottom-right, and bottom-left order
    return np.array([tl, tr, br, bl], dtype="float32")


In [9]:
# Define a function that takes an image, number of x and y points, 
# camera matrix and distortion coefficients
def corners_unwarp(img, corners, cameraMatrix, distCoeffs):
    # Use the OpenCV undistort() function to remove distortion
    undist = remove_distortion(img, cameraMatrix, distCoeffs)

    offset = 350 # offset for dst points
    
    # Grab the image shape
    img_size = (undist.shape[1], undist.shape[0])

    # For source points I'm grabbing the outer four detected corners
    src = np.float32(
                        [corners[0], ## BL
                         corners[1], ## TL
                         corners[2], ## BR
                         corners[3]  ## TR
                    ]) 
    # For destination points, I'm arbitrarily choosing some points to be
    # a nice fit for displaying our warped result 
    # again, not exact, but close enough for our purposes
    dst = np.float32([[offset, 0], ## Top Left
                      [img_size[0]-offset, 0], ## Top Right
                      [img_size[0]-offset, img_size[1]], ## Botom Right
                      [offset, img_size[1]] ## Bottom Left
                     ])
        
        
    # Given src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    
    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(undist, M, img_size)
    
    warped = lane_mask(warped, [np.array(dst, np.int32).reshape((-1,1,2))])
    # Return the resulting image and matrix
    return warped, M, Minv

In [10]:
def lane_mask(img, pts):
    #define a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel colour to fill the mask with depending on the input image
    if len(img.shape) == 3: 
        ignore_mask_colour = (0,255,0)
    else:
        ignore_mask_colour = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill colour    
    cv2.fillPoly(mask, pts, ignore_mask_colour)
    
    #returning the image only where mask pixels are nonzero
    #masked_image = cv2.bitwise_and(img, mask)
    return cv2.addWeighted(img,1.0,mask,0.2,0)

In [60]:

        
def get_new_lane(image, binary_warped, M, plot_on=True):
    global Left, Right
    #img_size = (image.shape[1], image.shape[0])
    #image_lane = preprocess(raw_image)
    #image_lane = np.dstack((image_lane, image_lane, image_lane))*255
    #warped_binary = cv2.warpPerspective(image_lane, M, img_size)
    #binary_warped = warped_binary.copy()[:,:,0]
    #warped_colour = cv2.warpPerspective(image, M, img_size)

    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255

    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 3) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 3) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    if PRINT_OUTS:
        print("Length of Left x,y, Right x,y", len(leftx), len(lefty), len(rightx), len(righty))
    
    if (len(leftx) == 0) or (len(lefty) == 0) or (len(rightx) == 0) or (len(righty) == 0):
        return None, None, None, None, None, None, None, None, None
    
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # get the best fit, by taking an average of previous best fits
    #left_fit = Left.get_best_fit(left_fit)
    #right_fit = Right.get_best_fit(right_fit)
    
    #return leftx,lefty,rightx,righty

    #def get_lane_fit_x(image,binary_warped,xvals,yvals)
    # Fit a second order polynomial
    #return np.polyfit(xvals, yvals, 2)
    
    #def get_lane_img(image, binary_warped, line_fit) 
    #    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    #    left_fitx = line_fit[0]*ploty**2 + line_fit[1]*ploty + line_fit[2]
    #    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    #    leftx_base = np.argmax(histogram[:midpoint])
    #    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    #    # Calculate the new radii of curvature
    #    y_eval = np.max(ploty)
    #    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    #    pts_left = np.array([np.transpose(np.vstack([left_fitx, yvals]))])
        
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    ym_per_pix = 40/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/(rightx_base-leftx_base) #lane_width_px # meters per pixel in x dimension
    centre = (leftx_base + rightx_base)/2
    position = (midpoint - centre)*xm_per_pix
    
    # Fit a second order polynomial to each
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)

    # Calculate the new radii of curvature
    y_eval = np.max(ploty)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])

    curvature = (left_curverad + right_curverad)/2
    
    yvals = ploty
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, yvals]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, yvals])))])
    pts = np.hstack((pts_left, pts_right))
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    # Combine the result with the original image
    result2 = cv2.addWeighted(image, 1, newwarp, 0.3, 0)
    # Put text on an image
    font = cv2.FONT_HERSHEY_SIMPLEX
    if curvature>=2500:
        text = "Radius of Curvature: Straight"
    else:
        text = "Radius of Curvature: {} m".format(int(curvature))
    cv2.putText(result2,text,(400,100), font, 1,(255,255,255),2)
    # Find the position of the car
    pts = np.argwhere(newwarp[:,:,1])
    
    if position < 0:
        text = "Vehicle is {:.2f} m left of centre".format(-position)
    else:
        text = "Vehicle is {:.2f} m right of centre".format(position)
    cv2.putText(result2,text,(400,150), font, 1,(255,255,255),2)
    
    result2 = cv2.cvtColor(result2, cv2.COLOR_BGR2RGB)
    
    # Draw the lane onto the warped blank image
    if plot_on == True:
        plt.figure(1, figsize=(12,6))
        #cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
        #cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
        #result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
        #result = cv2.addWeighted(image_lane, 1, result, 0.3, 0)
        #result2 = cv2.cvtColor(result2, cv2.COLOR_BGR2RGB)
        plt.imshow(result2)
        #plt.plot(left_fitx, ploty, color='yellow')
        #plt.plot(right_fitx, ploty, color='yellow')
        plt.xlim(0, 1280)
        plt.ylim(720, 0)

    return result2, left_fit, right_fit, left_curverad, right_curverad, left_fitx, right_fitx, yvals, position
    
    #return left_fit, right_fit

def plot_lane(image, binary_warped, left_curverad, right_curverad, left_fitx, right_fitx, yvals, position):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, yvals]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, yvals])))])
    pts = np.hstack((pts_left, pts_right))
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    # Combine the result with the original image
    result2 = cv2.addWeighted(image, 1, newwarp, 0.3, 0)
    
    # Put text on an image
    font = cv2.FONT_HERSHEY_SIMPLEX
    curvature = (left_curverad + right_curverad)/2
    if curvature>=2500:
        text = "Radius of Curvature: Straight"
    else:
        text = "Radius of Curvature: {} m".format(int(curvature))
    cv2.putText(result2,text,(400,100), font, 1,(255,255,255),2)
    # Find the position of the car
    pts = np.argwhere(newwarp[:,:,1])
    
    if position < 0:
        text = "Vehicle is {:.2f} m left of centre".format(-position)
    else:
        text = "Vehicle is {:.2f} m right of centre".format(position)
    cv2.putText(result2,text,(400,150), font, 1,(255,255,255),2)
    
    return result2
    
def get_next_lane(image, binary_warped, M, left_fit, right_fit, plot_on=True):
    global Left, Right
    
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
    left_fit[1]*nonzeroy + left_fit[2] + margin))) 

    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
    right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    if (len(leftx) == 0) or (len(lefty) == 0) or (len(rightx) == 0) or (len(righty) == 0):
        return None, None, None, None, None, None, None, None, None
    
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    if PRINT_OUTS:
        print("Left fit before:", left_fit) 
        print("Right fit before:", right_fit) 
        
    # get the best fit, by taking an average of previous best fits
    left_fit = Left.get_best_fit(left_fit)
    right_fit = Right.get_best_fit(right_fit)
    
    if PRINT_OUTS:
        print("Left fit after:", left_fit) 
        print("Right fit after:", right_fit) 
        
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                                  ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                                  ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    
    
    #return left_fitx, right_fitx, ploty
    #lane_width_px = rightx_base-leftx_base
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    ym_per_pix = 40/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/(rightx_base-leftx_base) #lane_width_px # meters per pixel in x dimension
    centre = (leftx_base + rightx_base)/2
    position = (midpoint - centre)*xm_per_pix
    
    # Fit a second order polynomial to each
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)

    # Calculate the new radii of curvature
    y_eval = np.max(ploty)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])

    curvature = (left_curverad + right_curverad)/2
    
    yvals = ploty
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, yvals]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, yvals])))])
    pts = np.hstack((pts_left, pts_right))
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    # Combine the result with the original image
    result2 = cv2.addWeighted(image, 1, newwarp, 0.3, 0)
    # Put text on an image
    font = cv2.FONT_HERSHEY_SIMPLEX
    if curvature>=2500:
        text = "Radius of Curvature: Straight"
    else:
        text = "Radius of Curvature: {} m".format(int(curvature))
    cv2.putText(result2,text,(400,100), font, 1,(255,255,255),2)
    # Find the position of the car
    pts = np.argwhere(newwarp[:,:,1])
    
    if position < 0:
        text = "Vehicle is {:.2f} m left of centre".format(-position)
    else:
        text = "Vehicle is {:.2f} m right of centre".format(position)
    cv2.putText(result2,text,(400,150), font, 1,(255,255,255),2)
    
    result2 = cv2.cvtColor(result2, cv2.COLOR_BGR2RGB)
    
    # Draw the lane onto the warped blank image
    if plot_on == True:
        plt.figure(1, figsize=(12,6))
        #cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
        #cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
        result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
        result = cv2.addWeighted(image_lane, 1, result, 0.3, 0)
        plt.imshow(result2)
        #plt.plot(left_fitx, ploty, color='yellow')
        #plt.plot(right_fitx, ploty, color='yellow')
        plt.xlim(0, 1280)
        plt.ylim(720, 0)
        
    return result2, left_fit, right_fit, left_curverad, right_curverad, left_fitx, right_fitx, yvals, position
    

In [61]:
def are_lines_parallel(line1_fit, line2_fit, threshold):
    first_order_diff = np.abs(line1_fit[0] - line2_fit[0])
    second_order_diff = np.abs(line1_fit[1] - line2_fit[1]) 

    if PRINT_OUTS:
        print("L1,L2, R1,R2: ", line1_fit[0], line1_fit[1], line2_fit[0], line2_fit[1]) 
        print("Parrallel Check (1st order diff, 2nd order diff): ",first_order_diff, second_order_diff) 
    return ((first_order_diff <= threshold[0]) & (second_order_diff <= threshold[1]))

# Test function
# print(are_lines_parallel([3,3,3],[4,4,4],(1.0,1.0)) )

def is_lane_width_correct(leftx, rightx, threshold):
    lane_width = np.abs(leftx - rightx)
    
    max_lane_width = np.amax(lane_width)
    min_lane_width = np.amin(lane_width)
    mav_lane_width = np.mean(lane_width)
    
    if PRINT_OUTS:
        print("Required min, max of lane width : ", threshold[0], threshold[1])  
        print("lane width (max,min,av): ", max_lane_width, min_lane_width, mav_lane_width)    
    return ((min_lane_width >= threshold[0]) & (max_lane_width <= threshold[1]))

# Test function
# print(are_lines_parallel([3,3,3],[4,4,4],(1.0,1.0)) )     

def is_curvature_valid(left_curverad, right_curverad, threshold):
    left_curverad = abs(left_curverad)
    right_curverad = abs(right_curverad)
    
    
    
    av = ((left_curverad+right_curverad)/2)
    diff = abs(left_curverad-right_curverad)
    if PRINT_OUTS:
            print("Left Curvature: ", left_curverad)
            print("Right Curvature: ", right_curverad)        
            print("Curvature Error between lanes: ", diff)
            print("Average Road Curvature: ",av)
            
    if left_curverad > 3000 or right_curverad > 3000:
        if PRINT_OUTS:
            print("Large curvature - almost straight")
        return True
    
    return ( (diff/av < 0.35) & (left_curverad >= threshold[1]) & (right_curverad >= threshold[1])) #& (diff <= threshold[0]))

In [62]:
def process_video(raw_image):
    global lane, Left, Right, mtx, dist, M, Minv, prev_frame
    
           
    # Take raw image and undistort it
    undst_img = remove_distortion(raw_image, mtx, dist)
    
    # Preprocess image and get binary
    processed_img = preprocess(undst_img)
    
    #processed_img_scaled = np.dstack((processed_img, processed_img, processed_img))*255
    img_size = (processed_img.shape[1], processed_img.shape[0])
    
    # Get warped binary image
    warped_binary = cv2.warpPerspective(processed_img, M, img_size)
    
    combined_warped_binary = lane.get_combined_lane_img(warped_binary)
    
    if (Left.detected == False) | (Right.detected == False):
        result, left_fit, right_fit, left_curverad, right_curverad, left_fitx, right_fitx, yvals, position = get_new_lane(undst_img, combined_warped_binary, M, plot_on=False)
    else:
        result, left_fit, right_fit, left_curverad, right_curverad, left_fitx, right_fitx, yvals, position = get_next_lane(undst_img, combined_warped_binary, M, Left.best_fit, Right.best_fit, plot_on=False)
   
    result = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)

    if PRINT_OUTS:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
        ax1.imshow(combined_warped_binary)
        ax1.set_title('Combined Binary', fontsize=30)
        ax2.imshow(result)
        plt.show()
        
    ### Do some sanity checks then register whether the lines have been detected and are sane
    # We'll start by assuming that we found the lines
    lanes_detected = True
    
    if result == None:
        if PRINT_OUTS:
            print("FAILED: Result was empty")
        lanes_detected = False
        Right.detected = False
        Left.detected = False
        if prev_frame == []:
            if PRINT_OUTS:
                print("RESULT IMAGE NOT VALID: Returning blank frame")
            return undst_img
        else:
            if PRINT_OUTS:
                print("RESULT IMAGE NOT VALID: Returning previous frame")
            prev_frame = plot_lane(undst_img, combined_warped_binary,Left.radius_of_curvature, Right.radius_of_curvature, Left.allx, Right.allx, Left.ally, Left.line_base_pos)
            return prev_frame
 
    # Are the lanes parallel?
    parallel = are_lines_parallel(left_fit, right_fit, PARALLEL_THRESHOLD)
    if parallel == False:
        lanes_detected = False
        if PRINT_OUTS:
            print("LINES NOT VALID: NOT PARALLEL")

    # Are the lanes the correct width?
    correct_width = is_lane_width_correct(left_fitx, right_fitx, LANE_WIDTH_THRESHOLD)
    if correct_width == False:
        lanes_detected = False
        if PRINT_OUTS:
            print("LINES NOT VALID: NOT CORRECT WIDTH")

    # Is the road curvature valid
    correct_curvature = is_curvature_valid(left_curverad, right_curverad, CURVATURE_THRESHOLD)
    if correct_curvature == False:
        lanes_detected = False
        if PRINT_OUTS:
            print("LINES NOT VALID: CURVATURE INVALID")
    
    if lanes_detected == False:
        if prev_frame == []:
            if PRINT_OUTS:
                print("LINES NOT VALID: Returning blank frame")
            return undst_img
        else:
            if PRINT_OUTS:
                print("LINES NOT VALID: Returning previous frame")
            prev_frame = plot_lane(undst_img, combined_warped_binary, Left.radius_of_curvature, Right.radius_of_curvature, Left.allx, Right.allx, Left.ally, Left.line_base_pos)
            return prev_frame

    # We have a new valid line, report the line is detected and update lines
    #if lanes_detected == True:
    Right.update_line(lanes_detected, right_fit, right_curverad, position, right_fitx, yvals)
    Left.update_line(lanes_detected, left_fit, left_curverad, position, left_fitx, yvals)
    if PRINT_OUTS:
        print("LINES VALID: Lines updated and saving and returning current frame.")
    prev_frame = result
    return result   
 
    
    #Right.detected = False
    #Left.detected = False
    #return undst_img
       
    
   
    

In [63]:
import imageio
imageio.plugins.ffmpeg.download() # you may need to install, try run this

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [67]:
%matplotlib inline
PRINT_OUTS = False

prev_frame = []
ret, mtx, dist, rvecs, tvecs = fetch_camera_calibration()
M, Minv = fetch_perspective_transform()
Left = Line(n_fits=8)
Right = Line(n_fits=8)
lane = Lane(n_fits=8) # averaging of the binary frame

video_output = 'challenge_result1.mp4'
#video_output = 'challenge_result.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
clip = VideoFileClip('challenge_video.mp4').subclip(38,46)
#clip = VideoFileClip('challenge_video.mp4').subclip(1,2)
video_clip = clip.fl_image(process_video)
%time video_clip.write_videofile(video_output, audio=False)
   
clip.reader.close()
video_clip.reader.close()
video_clip.audio.reader.close_proc()

error: ..\..\..\modules\imgproc\src\color.cpp:7341: error: (-215) scn == 3 || scn == 4 in function cv::ipp_cvtColor
